In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
import zipfile
import io
import xlsxwriter
import time

In [14]:
#Funções:

###Função requisicao dados CVN
# Função para fazer a requisição e tratar os possíveis erros
def fazer_requisicao(url,mes ,ano):
    try:
        # Realizar a requisição
        response = requests.get(url)

        # Verificar o status code da resposta
        if response.status_code == 200:
            print('Requisição bem-sucedida para '+mes+'/'+ano+'.')
            return response
        elif response.status_code == 404:
            print('Página não encontrada (erro 404). Verifique a devida publicação na CVM disponível no site https://dados.cvm.gov.br/dados/FI/DOC/')
            return  response # Interrompe o código aqui se o status code for 404
        else:
            print(f'Erro inesperado: Status code {response.status_code}')
            return  # Interrompe o código em caso de status code diferente de 200 e 404

    except requests.ConnectionError:
        print('Erro de conexão: Verifique sua conexão de internet.')
    except requests.Timeout:
        print('Erro de timeout: A requisição excedeu o tempo limite.')
    except requests.RequestException as e:
        print(f'Erro na requisição: {e}')


def coleta(objetivo, mes, ano):
    
    """
    Esta função captura dados do site da CVM.

    Parâmetros:
    objetivo(string): qual o tipo de informação deseja 'lamina' ou 'informativo'.
    mes(int): qual o mês do dado.
    ano(int): qual o ano do dado.
    
    Retorno:

    df com o dado CVM 
    
    
    """
    objetivo = objetivo
    
    ano = str(ano)
    mes = str(mes).zfill(2) 
       
    informativo_dia  = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_'+ano+mes+'.zip'
    lamina =  'https://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/lamina_fi_'+ano+mes+'.zip'
    arquivo_inf_diario = 'inf_diario_fi_'+ano+mes+'.csv'
    arquivo_lamina = 'lamina_fi_'+ano+mes+'.csv'
    

    if objetivo =='informativo':
        link_arq = informativo_dia
        tp_arq = arquivo_inf_diario
    elif objetivo =='lamina':
        tp_arq = arquivo_lamina
        link_arq = lamina

    print('{}\n{}'.format(link_arq, tp_arq))
    
    r = fazer_requisicao(link_arq,mes ,ano)
    
    if r.status_code == 404:
        print()
        
    else:

        zf = zipfile.ZipFile(io.BytesIO(r.content))

        arquivo = tp_arq

        zf = zf.open(arquivo)

        lines = zf.readlines()
        lines =[i.strip().decode('ISO-8859-1') for i in lines]
        lines = [i.split(';') for i in lines]

        df = pd.DataFrame(lines[1:], columns = lines[0])
        return df
    

def cadastro_cvm(tipo_fundo = 'FI', situacao = 'EM FUNCIONAMENTO NORMAL'):
    """
    Função busca dados cadastrais CVM

    Parametros:

    tipo_fundo (string or list): por padrão 'FI', Fundo de Investimento mas pode ser qualquer um dessa lista
    ['FACFIF', 'FAPI', 'FCCE', 'FCCE-AI', 'FI', 'FI-FGTS', 'FIC-FITVM',
       'FICART', 'FICE', 'FIDC', 'FIEX', 'FIF', 'FIFDIV', 'FII', 'FIIM',
       'FIP', 'FIQ-FMIA', 'FITVM', 'FMAI', 'FMIA', 'FMIA-CL', 'FMIEE',
       'FMP-FGTS', 'FMP-FGTS CL', 'FPCE', 'FPDS', 'FUNCINE']


    situação (string or list): por padrão 'EM FUNCIONAMENTO NORMAL', mas pode ser qualquer da lista 
    ['CANCELADA', 'EM FUNCIONAMENTO NORMAL', 'LIQUIDAÇĂO',
       'FASE PRÉ-OPERACIONAL', 'EM ANÁLISE', 'INCORPORAÇĂO',
       'EM SITUAÇĂO ESPECIAL']
    """
    r = requests.get('https://dados.cvm.gov.br/dados/FI/CAD/DADOS/cad_fi.csv')
    cad_cvm = pd.read_csv(io.StringIO(r.text), delimiter=';')
    if isinstance(tipo_fundo,str):
      lista_tipo_fundo = [tipo_fundo]
    else:
      lista_tipo_fundo = tipo_fundo

    if isinstance(situacao,str):
      lista_situacao = [situacao]
    else:
      lista_situacao = situacao
    df = cad_cvm.loc[(cad_cvm['SIT'].isin(lista_situacao)) & (cad_cvm['TP_FUNDO'].isin(lista_tipo_fundo))]
    return df


In [15]:
df_l = coleta('lamina',2,2024)
df_inf = coleta('informativo',1,2024)

https://dados.cvm.gov.br/dados/FI/DOC/LAMINA/DADOS/lamina_fi_202402.zip
lamina_fi_202402.csv
Requisição bem-sucedida para 02/2024.
https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202401.zip
inf_diario_fi_202401.csv
Requisição bem-sucedida para 01/2024.


In [16]:
df_inf.tail(3)

,TP_FUNDO,CNPJ_FUNDO,DT_COMPTC,VL_TOTAL,VL_QUOTA,VL_PATRIM_LIQ,CAPTC_DIA,RESG_DIA,NR_COTST
567671,FI,97.929.213/0001-34,2024-01-29,87209656.10,11.631119520000,83606058.33,0.00,0.00,2
567672,FI,97.929.213/0001-34,2024-01-30,85912226.51,11.610815220000,83460108.29,0.00,0.00,2
567673,FI,97.929.213/0001-34,2024-01-31,85913618.59,11.610931950000,83460947.30,0.00,0.00,2


In [17]:
df_l.tail(3)


,CNPJ_FUNDO,DENOM_SOCIAL,DT_COMPTC,NM_FANTASIA,ENDER_ELETRONICO,PUBLICO_ALVO,RESTR_INVEST,OBJETIVO,POLIT_INVEST,PR_PL_ATIVO_EXTERIOR,...,VL_DESPESA_3ANO,VL_DESPESA_5ANO,VL_RETORNO_3ANO,VL_RETORNO_5ANO,REMUN_DISTRIB,DISTRIB_GESTOR_UNICO,CONFLITO_VENDA,TEL_SAC,ENDER_ELETRONICO_RECLAMACAO,INF_SAC
5517,97.261.093/0001-40,BANRISUL PERFORMANCE FUNDO DE INVESTIMENTO EM ...,2024-02-29,BANRISUL PERFORMANCE FUNDO DE INVESTIMENTO EM ...,www.banrisulcorretora.com.br,"Investidores em geral, inclusive Regimes Própr...",,Proporcionar aos cotistas a rentabilidade de s...,"A gestão de recursos do Fundo é ativa, em que ...",0.000000,...,91.200000,168.220000,239.790000,442.280000,As cotas do Fundo são distribuídas exclusivame...,"O distribuidor oferta, para o público alvo do ...",,5132152300,"OUVIDORIA 08006442200, SAC 08006461515",Para os investidores que já possuem outros inv...
5518,97.519.794/0001-36,ITAÚ REFERENCIADO DI RENDA FIXA EVOLUÇÃO FUNDO...,2024-02-29,ITAÚ REFERENCIADO DI RENDA FIXA,www.itaupersonnalite.com.br,"O Fundo, a critério do Administrador, receberá...",NÃO HÁ RESTRIÇÃO,Acompanhar a variação do CDI por meio da aplic...,O objetivo do FUNDO é aplicar seus recursos em...,0.000000,...,15.620000,26.170000,315.380000,584.340000,Os distribuidores são contratados pelos própri...,"O Itaú Unibanco, principal distribuidor em seu...",O Itaú Unibanco distribui preponderante fundos...,8007227377,www.itau.com.br,SUBSTITUA ESTA FRASE PELAS INFORMAÇÕES ADICION...
5519,97.548.167/0001-23,JGP STRATEGY MASTER FUNDO DE INVESTIMENTO MULT...,2024-02-29,JGP STRATEGY MASTER FI MULTIMERCADO,https://servicosfinanceiros.bnymellon.com,O FUNDO tem como público alvo fundos de invest...,,O objetivo do FUNDO é proporcionar a valorizaç...,A política de investimento adotada pelo FUNDO ...,20.000000,...,22.950000,42.400000,308.050000,568.110000,Os Distribuidores são remunerados diretamente ...,Sim,,8007253219,"21 3219-2600,0800-7253219 ou sac@bnymellon.com.br",


In [18]:
cadastro_cvm().tail(3)

C:\Users\Pichau\AppData\Local\Temp\ipykernel_12572\1475115120.py:105: DtypeWarning: Columns (14,17,18,20,22,24,27,37,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  cad_cvm = pd.read_csv(io.StringIO(r.text), delimiter=';')


,TP_FUNDO,CNPJ_FUNDO,DENOM_SOCIAL,DT_REG,DT_CONST,CD_CVM,DT_CANCEL,SIT,DT_INI_SIT,DT_INI_ATIV,...,CPF_CNPJ_GESTOR,GESTOR,CNPJ_AUDITOR,AUDITOR,CNPJ_CUSTODIANTE,CUSTODIANTE,CNPJ_CONTROLADOR,CONTROLADOR,INVEST_CEMPR_EXTER,CLASSE_ANBIMA
59550,FI,97.711.801/0001-05,953 FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS D...,2011-10-04,2011-07-07,169390.0,NaN,EM FUNCIONAMENTO NORMAL,2011-10-07,2011-10-07,...,10.639.564/0001-22,WRIGHT CAPITAL GESTAO DE RECURSOS LTDA,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,60.701.190/0001-04,ITAU UNIBANCO S.A.,60.701.190/0001-04,ITAU UNIBANCO S.A.,S,Multimercados Invest. no Exterior
59555,FI,97.929.197/0001-80,LECT FUNDO DE INVESTIMENTO EM COTAS DE FUNDOS ...,2011-08-31,2011-08-07,167991.0,NaN,EM FUNCIONAMENTO NORMAL,2011-11-04,2011-11-04,...,10.957.035/0001-77,ATMOS CAPITAL GESTĂO DE RECURSOS LTDA,61.366.936/0001-25,ERNST & YOUNG AUDITORES INDEPENDENTES S/S LTDA.,30.306.294/0001-45,BANCO BTG PACTUAL S/A,59.281.253/0001-23,BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM,N,Açơes Livre
59556,FI,97.929.213/0001-34,WHG SISTEMA II FUNDO DE INVESTIMENTO MULTIMERC...,2011-08-31,2011-08-07,168017.0,NaN,EM FUNCIONAMENTO NORMAL,2011-11-18,2011-11-18,...,34.848.969/0001-39,WEALTH HIGH GOVERNANCE CAPITAL LTDA.,61.562.112/0001-20,PRICEWATERHOUSECOOPERS AUDITORES INDEPENDENTES...,62.318.407/0001-19,S3 CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VA...,62.318.407/0001-19,S3 CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VA...,S,Multimercados Invest. no Exterior
